# Open Rocket Engine Development Notebook

This notebook will temporarily be used to calculate engine parameters. Equations, classes, etc. built here will eventually be moved to the enginebuilder module

In [13]:
import matplotlib.pyplot as plt
import numpy as np

In [18]:
def atm_to_psi(pressure):
    return 14.6959*pressure


class Rocket():
    def __init__(self):
        pass
    
    def get_designParamters(self):
        """ Thrust Chamber Design Parameters """
        # Using P_exit = 1, and plots from
        # http://www.braeunig.us/space/comb-OM.htm P_Chamber, T_Chamber, y, Gas_Molecular_Weight can be extracted
        # after choosing the mixture ratio.
        self.Characteristic_Length = 40;    # units - inches; interpolated use table 4-1 from Huzel+Huang Modern Engineering for Design of LRES 
        self.Thrust = 2000;                 # units - pounds
        self.Tc = 2900.3;              # Chamber temperature; units - Celcius
        self.Pc = 20;                       # Chamber pressure; units - atm
        self.Pe = 0.82206329;               # Exit pressure - sea level; units - atm
        self.Pa = .9278;                    # Ambient pressure - sea level; units - atm
        self.Mixture_Ratio = 2.62;          # O/F mass ratio; unitless
        self.Gas_Molecular_Weight = 18.833; # units - amu
        self.gamma = 1.1510;                    # Ratio of Specific Heats - unitless
        self.Working_Stress = 80000;        # units - psi; yield stress of inconel walling
        
    def get_constants(self):
        """ Define constants """
        self.G_EARTH =  32.1740;   # units - ft/seconds^2
        self.R = 1545.348963;      # universal gas constant; units - ft*lbf/R*lb-mol
        self.FOS = 1.4;    # Factor of Safety

    def get_temperatures(self):
        """ Adiabatic Flame Temperature - Chamber Temperature; CONVERSIONS """
        self.TcK = self.Tc+273.15;  # units - Kelvin
        self.TcF = self.Tc*1.8+32;  # units - Fahrenheit
        self.TcR = self.TcF+459.67;  # units - Rankine

    def get_expansion(self):
        """ Expansion Ratio """
        self.expansionRatio = ((2/(self.gamma+1))**(1/(self.gamma-1))*(self.Pc/self.Pe)**(1/self.gamma)/
                           ((self.gamma+1)/(self.gamma-1)*(1 - (self.Pe/self.Pc)**(1-1/self.gamma)) )**.5)  # unitless

    def get_cStar(self):
        """ Charateristic Velocity """
        self.ceeStar = (self.G_EARTH*self.gamma*self.R*self.TcR/self.Gas_Molecular_Weight)**(.5)/(self.gamma*((2/(self.gamma+1))^((self.gamma+1)/(self.gamma-1)))^.5)  # ft/second

    def get_correctionFactor(self):
        """ correction factor """
        self.lambaFactor = 0.985
        
    def get_thrustCoefficient(self):
        """ Thrust Coefficient"""
        self.Cf = self.lamdaFactor*(((2*self.gamma*self.gamma)/(self.gamma-1)*(2/(y+1))**((y+1)/(y-1))*
                                     (1-(Pe/Pc)**((self.gamma-1)/self.gamma)))**(1/2) + self.expansionRatio*(self.Pe-self.Pa)/self.Pc)

    def get_Isp(self):
        """ Specific Impulse """
        self.Isp = self.cStar*self.Cf/self.G_EARTH  # units - seconds

    def get_mdot(self):
        """ Mass Flow Rates """
        self.mdotTotal = self.Thrust/self.Isp                # units - lbs/s
        self.mdotF = self.mdotTotal/(1+self.Mixture_Ratio)   # units - lbs/s
        self.mdotO = self.mdotTotal - mdotF                  # units - lbs/s

In [ ]:
class TCADimensions():
    def __init__(self):
        pass
    
    def get_NozzleThroatParameters(self):
         """Nozzle Throat Parameters"""
        T_Throat_R = T_Chamber_R*(2/(1+y))                            % units - Rankine
        P_Throat = P_Chamber*(y/2 +.5)^(y/(1-y))                      % units - atm
        Area_Throat = (Massflow_Total*Isp) / (P_Chamber*atm_to_psi*Cf)% units - in^2
        Diameter_Throat = (Area_Throat/pi)^.5 * 2            % units - in

    def get_ContractionAreaRatio(self):
        """Contraction Ratio"""
        Contraction_Ratio = 1.317303 + (32346880 - 1.317303)/(1 + (Diameter_Throat/1.041567e-10)^0.6774809) % units - inches;
        # pulled from figure 4-9 of Modern Engineering for Design of LRES 
        # fitted using https://mycurvefit.com/

    def get_NozzleExit(self):
        """Nozzle Exit"""
        self.Area_Exit = Expansion_Ratio * Area_Throat % units - in^2
        Diameter_Exit = (Area_Exit/pi)^.5 * 2 % units - in

%% Chamber Parameters
Volume_Chamber = Area_Throat*Characteristic_Length % units - in^3
Area_Chamber = Area_Throat*Contraction_Ratio       % units - in^2
Diameter_Chamber = (Area_Chamber/pi)^.5 * 2        % units - in^1
Length_Chamber = Volume_Chamber/Area_Chamber/1.1   % units - in^1

%% Wall Thickness
Thickness_ChamberWall = Factor_of_Safety*P_Chamber*atm_to_psi*Diameter_Chamber / (2*Working_Stress - 2*Factor_of_Safety*P_Chamber*atm_to_psi) %units - in^1

%% Nozzle Design
% y = Px+Q+(S*x+T)^(1/2)
% Curve Fitting Conditions  
%     1) y(x0 == 0) == y0 == 0
%     2) y(x1 == Nozzle_Length  - .382*Radius_Throat*sin(Theta_Initial) )
%     == y1 == Radius_exit - (.382*Radius_Throat*(1-cos(Theta_Initial)) +
%     Radius_Throat)
%     3) y'(x0 == 0) == tan(theta_initial)
%     4) y'(x1 == Nozzle_Length  - .382*Radius_Throat*sin(Theta_Initial)) == tan(theta_exit)
 
% http://soliton.ae.gatech.edu/people/jseitzma/classes/ae6450/bell_nozzle.pdf
% Sections 3.4+ in https://www.ewp.rpi.edu/hartford/~ernesto/S2013/EP/MaterialsforStudents/Lee/Sutton-Biblarz-Rocket_Propulsion_Elements.pdf
Radius_Throat = Diameter_Throat/2;
Radius_Exit = Diameter_Exit/2;

Theta_Initial = 25*3.141592/180; % Given in degrees by Rao, based on expansion ratio; here it's converted to radians
Theta_Exit = 12*3.141592/180; % Given in degrees by Rao, based on expansion ratio; here it's converted to radians

% (xN,yN) is the point where circular approximation ends and parabolic
% approximation begins; relative to center of nozzle throat
xN = .382*Radius_Throat*sin(Theta_Initial);                         %units inches
yN = .382*Radius_Throat*(1-cos(Theta_Initial)) + Radius_Throat;     %units inches

% (x1,y1) exit lip of the nozzle relative to (xN,yN)
x1 = .8*(Diameter_Throat/2)/tan(15*pi/180)*(Expansion_Ratio^.5-1+1.5*(1/cos(15*pi/180)-1)) - xN;%units inches
y1 = Radius_Exit -yN;                                           %units inches

N = tan(Theta_Initial); %Derivative at (x0,y0)
E = tan(Theta_Exit);    %Derivative at (x1,y1)

P = (E*(2*x1*N-y1) - y1*N)/(x1*N+x1*E-2*y1)
Q = -(y1-x1*E)^2*(x1*N-y1)/(2*(.5*x1*E+.5*x1*N-y1)^2)
S = -(y1-x1*E)^2*(y1-x1*N)^2*(E-N)/(2*(.5*x1*E+.5*x1*N-y1)^3)
T = (y1-x1*E)^4*(y1-x1*N)^2/(4*(.5*x1*E+.5*x1*N-y1)^4)
